In [1]:
!pip install transformers[sentencepiece]

In [2]:
cd ../input/dataset-of-df

/kaggle/input/dataset-of-df


In [3]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import pandas as pd
import spacy
review = pd.read_csv("df_app_reviews.csv")
iss = pd.read_csv("df_app_issue.csv",encoding="ISO-8859-1")
model = T5EncoderModel.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.4.layer.0.SelfAttention.v.weight', 'decoder.block.5.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.5.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.5.layer.2.DenseReluDense.wo.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.5.layer.0.SelfAttention.o.weight', 'decoder.block.4.layer.0.SelfAttention.o.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.5.layer.0.SelfAttention.v.weight', 'decoder.block.2.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'de

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [4]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2022-04-15 13:13:01.154814: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-15 13:13:01.170079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 13:13:01.171410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 13:13:01.172177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14608908713495367101,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15308947456
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13833413241001219553
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [5]:
feedback = review.nextcloud.unique()
issue = iss.nextcloud.unique()

In [6]:
len(feedback)

50

In [7]:
def spacy_fenxi(words):
    nlp = spacy.load("en_core_web_sm")
#     nlp = spacy.load('en')
    doc = nlp(words)
    l = len(doc)
#     print(len(doc))
    # print(doc)
#     print(type(doc))
    list1 = [0 for i in range(l)] #初始化全0列表
    # print(type(doc)) #<class 'spacy.tokens.doc.Doc'>
    i=0
    list2 = []

    for token in doc:
#         print(token, token.pos_, token.pos, token.tag, token.tag_)
        if(len(str(token))!=1):
          list2.append('▁'+str(token))
        else:
          list2.append(str(token))
        if(token.pos == 92 or token.pos == 100):
          list1[i] = 1 #名词动词设为1
          i = i+1
        else:
          i = i+1
    return list1,list2

In [8]:
subword = ''
def t5_bitmask(words):
    global subword
    # tokenize = T5Tokenizer.tokenize(txt)
    inputs2 = tokenizer(words, return_tensors="pt").input_ids
    tokens2 = tokenizer.tokenize(words)
    l = len(tokens2)

    list3 = [0 for i in range(0,l)]
    i=0
    j=0

    #tokens列表  标bitmask
    while i < len(list1) :
        if(list1[i] == 1):
            # print([list2[i]])
            j=i
            token = tokens2[j]
            if(list2[i] == token):
                list3[j] = 1
                i+=1
                j+=1
            else:
                #先看没有拆分的话找一找
                m=i
                n=0
                flag = 0  # 标志是否找到
                while n < len(tokens2):
                    # print(list2[m],tokens2[n])
                    if (list2[m] == tokens2[n] and flag == 0):
                        list3[n] = 2
                        i+=1
                        j=n
                        flag=1
                        break
                        # m += 1
                    else:
                        n += 1
                #拆分了的话再拼接着找一遍，
                if(flag==0):  # 上一个循环没找到，代表拆分了.用是否包含子字符串的方法 in
                    j=0
                    # print("list2[i]:", list2[i], "tokens2[j]:", tokens2[j], "tokens2[j + 1]:", tokens2[j + 1])
                    #先找子字符串
                    while(j<len(tokens2)-1):
                        if (tokens2[j] in list2[i] and tokens2[j+1] in list2[i] and tokens2[j][0] == '▁') : #找到以下划线开头的子字符串
                            subword = tokens2[j]
                            list3[j] = 1
                            break
                        else:
                            j+=1
                    # subword = tokens2[j]
                    while(subword != list2[i] and j<len(tokens2)-1):
                        
                        j=j+1
                        subword +=tokens2[j]
                        list3[j] = 1
                    flag = 1
                    i+=1
                else:
                    i+=1
        else:
            i+=1
    return list3



In [9]:
import torch.nn as nn
class ConineSimilarity(nn.Module):

    def forward(self, tensor_1,tensor_2):
        normalized_tensor_1=tensor_1/tensor_1.norm(dim=-1,keepdim=True)
        normalized_tensor_2=tensor_2/tensor_2.norm(dim=-1,keepdim=True)
        return (normalized_tensor_1*normalized_tensor_2).sum(dim=-1)

con = ConineSimilarity()

In [10]:
def getvec(words):
  inputs = tokenizer(words, return_tensors="pt").input_ids
  outputs = model(inputs)
  last_hidden_state = outputs.last_hidden_state
  pooled_sentence = torch.mean(last_hidden_state, dim=1)

  return pooled_sentence

def search(words):
    CS1 = con(pingjun,getvec(issue[0]))
    CS2 = con(pingjun,getvec(issue[1]))
    CS3 = con(pingjun,getvec(issue[2]))
    Iss1 = issue[0]
    Iss2 = issue[1]
    Iss3 = issue[2]
#     print(CS1,CS2,CS3)
    max1 = max2 = max3=0
    #先将前三个的相似度按顺序排好为max1,max2,max3
    if(CS1>=CS2 and CS1>=CS3):
      max1 = CS1
      Iss1 = issue[0]
      if(CS2 >= CS3):
        max2 = CS2
        Iss2 = issue[1]
        max3 = CS3
        Iss3 = issue[2]
      else:
        max2 = CS3
        Iss2 = issue[2]
        max3 = CS2
        Iss3 = issue[1]
    elif(CS2>=CS3 and CS2>=CS1):
      max1 = CS2
      Iss1 = issue[1]
      if (CS3 >= CS1):
        max2 = CS3
        Iss2 = issue[2]
        max3 = CS1
        Iss3 = issue[0]
      else:
        max2 = CS1
        Iss2 = issue[0]
        max3 = CS3
        Iss3 = issue[2]
    elif(CS3>=CS2 and CS3>=CS1):
      max1 = CS3
      Iss1 = issue[2]
      if (CS2 >= CS1):
        max2 = CS2
        Iss2 = issue[1]
        max3 = CS1
        Iss3 = issue[0]
      else:
        max2 = CS1
        Iss2 = issue[0]
        max3 = CS2
        Iss3 = issue[1]
    i=3 #计数器，从第三条开始计算
    # 在比较剩下的和max1,max2,max3的大小
    for iss in issue[3:]:
      issvec = getvec(str(iss))
      CS = con(pingjun,issvec)
      if(CS > max3 and CS < max2):
        max3 = CS
        Iss3 = issue[i]
      elif(CS >max3 and CS > max2 and CS < max1):
        max3 = max2
        Iss3 = Iss2
        max2 = CS
        Iss2 = issue[i]
      elif(CS > max1):
        max3 = max2
        Iss3 = Iss2
        max2 = max1
        Iss2 = Iss1
        max1 = CS
        Iss1 = issue[i]

      i=i+1

#     print(max1,Iss1,max2,Iss2,max3,Iss3)
    return words,max1,Iss1,max2,Iss2,max3,Iss3

In [11]:
sentence = "Not working on Pixel 4 XL stock Android 10, won't.let me log in. Provider is Google FI (not Fiber) with automatic VPN.......is that the issue?"
list1,list2 = spacy_fenxi(sentence)
list3 = t5_bitmask(sentence)
#bug report用名词位置的向量表示
list3.append(0) #结尾有个</s> 要和last相乘
noun_tensor = torch.tensor(list3).reshape(1,len(list3),1)
# print(noun_tensor)
input_ids = tokenizer(sentence, return_tensors="pt").input_ids  # Batch size 1
outputs = model(input_ids=input_ids)
last_hidden_states = outputs.last_hidden_state
#     print(last_hidden_states)
res = noun_tensor * last_hidden_states
#     print(res)
#     print(res.shape)
pingjun = torch.mean(res,dim=1)
# pingjun = torch.mean(pingjun,dim=0)

In [12]:
words,max1,Iss1,max2,Iss2,max3,Iss3 =search("Not working on Pixel 4 XL stock Android 10, won't.let me log in. Provider is Google FI (not Fiber) with automatic VPN.......is that the issue?")
f = []
i = []
d = []

f.append(words)
f.append(words)
f.append(words)
i.append(Iss1)
i.append(Iss2)
i.append(Iss3)
d.append(max1.detach().numpy()[0])
d.append(max2.detach().numpy()[0])
d.append(max3.detach().numpy()[0])

print(f)
print(i)
print(d)

["Not working on Pixel 4 XL stock Android 10, won't.let me log in. Provider is Google FI (not Fiber) with automatic VPN.......is that the issue?", "Not working on Pixel 4 XL stock Android 10, won't.let me log in. Provider is Google FI (not Fiber) with automatic VPN.......is that the issue?", "Not working on Pixel 4 XL stock Android 10, won't.let me log in. Provider is Google FI (not Fiber) with automatic VPN.......is that the issue?"]
["Huawei mate 9 I can't get the automatic synchronization to work The problem of optimizing the battery remains on the screen even if it is set as not allowed The problem will be in emui 9.1.0 because for other mobile phone it works, how can I be helped for this bug  ?", 'Can\'t use the app anymore after update to 3.3: "Wrong login details"', 'Android (playstore) app will not log in to server 12.04 backend']
[0.56580764, 0.55934674, 0.5555978]


In [13]:
f = []
i = []
d = []

feedback = review.vlc.unique()
issue = iss.vcl.unique()

for fb1 in feedback:
  # CS1 = CS2 = CS3 = 0
  # Iss1 = Iss2 = Iss3 = ""
    sentence = fb1
    list1,list2 = spacy_fenxi(sentence)
    list3 = t5_bitmask(sentence)
    #bug report用名词位置的向量表示
    list3.append(0) #结尾有个</s> 要和last相乘
    noun_tensor = torch.tensor(list3).reshape(1,len(list3),1)
    # print(noun_tensor)
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids  # Batch size 1
    outputs = model(input_ids=input_ids)
    last_hidden_states = outputs.last_hidden_state
    #     print(last_hidden_states)
    res = noun_tensor * last_hidden_states
    #     print(res)
    #     print(res.shape)
    pingjun = torch.mean(res,dim=1)

    words,max1,Iss1,max2,Iss2,max3,Iss3 =search(fb1)
    f.append(words)
    f.append(words)
    f.append(words)
    i.append(Iss1)
    i.append(Iss2)
    i.append(Iss3)
    d.append(max1.detach().numpy()[0])
    d.append(max2.detach().numpy()[0])
    d.append(max3.detach().numpy()[0])


In [14]:
cd /kaggle/working

/kaggle/working


In [15]:
data = pd.DataFrame({'feedback':f,'issue':i,'distance':d})
data.to_csv("vlc_result_bitmask-1-2.csv",index=False)